# Leaderboard admin helper

Simple notebook to view and manage the 67 sprint leaderboards via the Next.js API.

- Lists **new (balanced)** and **old** sprint leaderboards
- Lets you **edit** or **delete** entries using the admin API key from `.env`

Before using this notebook:
- Make sure the Next.js app is running (e.g. `npm run dev` → usually `http://localhost:3000`).
- Ensure `.env` in this folder contains `LEADERBOARD_ADMIN_KEY` (already set for this project).

Run the cells below in order.

In [1]:
import os
from datetime import datetime
from typing import Literal, Optional, Dict, Any, List

import requests

# Base URL of the Next.js app. Change this if you deploy elsewhere.
BASE_URL = os.getenv("LEADERBOARD_BASE_URL", "http://localhost:3000")

def load_env_value(name: str, path: str = ".env") -> str:
  """Load a value from environment or from a simple .env file in this folder."""
  val = os.getenv(name)
  if val:
    return val
  try:
    with open(path, "r", encoding="utf-8") as f:
      for line in f:
        line = line.strip()
        if not line or line.startswith("#"):
          continue
        if line.startswith(name + "="):
          return line.split("=", 1)[1]
  except FileNotFoundError:
    pass
  raise RuntimeError(f"{name} not found in environment or {path}")

ADMIN_KEY = load_env_value("LEADERBOARD_ADMIN_KEY")



In [2]:
print(f"Using BASE_URL={BASE_URL}")
print("Loaded LEADERBOARD_ADMIN_KEY from environment/.env")


Using BASE_URL=http://localhost:3000
Loaded LEADERBOARD_ADMIN_KEY from environment/.env


In [3]:
Kind = Literal["new", "old"]

def _url(path: str) -> str:
  return BASE_URL.rstrip("/") + path

def get_leaderboard(kind: Kind = "new") -> List[Dict[str, Any]]:
  """Fetch leaderboard entries for the given kind ("new" or "old")."""
  resp = requests.get(_url("/api/leaderboard"), params={"kind": kind})
  resp.raise_for_status()
  data = resp.json()
  return data.get("entries", [])

def print_leaderboard(entries: List[Dict[str, Any]]) -> None:
  """Pretty-print leaderboard entries with IDs so you can edit/delete."""
  if not entries:
    print("No entries.")
    return
  for e in entries:
    ts = datetime.fromtimestamp(e["createdAt"] / 1000).isoformat(timespec="seconds")
    print(
      f"id={e['id']:3d} | score={e['score']:3d} | name=\"{e['name']}\" | createdAt={ts} | kind={e.get('kind', '?')}"
    )

def edit_entry(
  entry_id: int,
  *,
  name: Optional[str] = None,
  score: Optional[int] = None,
  kind: Kind = "new",
) -> Dict[str, Any]:
  """Edit a leaderboard entry. You must provide both name and score."""
  if name is None or score is None:
    raise ValueError(
      "Provide both name and score for edit_entry (keep existing values if you don't want to change them)."
    )
  payload = {
    "adminKey": ADMIN_KEY,
    "id": int(entry_id),
    "name": name,
    "score": int(score),
    "kind": kind,
  }
  resp = requests.patch(_url("/api/leaderboard"), json=payload)
  print("Status:", resp.status_code)
  data = resp.json()
  print(data)
  return data

def delete_entry(entry_id: int) -> Dict[str, Any]:
  """Delete a leaderboard entry by id."""
  payload = {"adminKey": ADMIN_KEY, "id": int(entry_id)}
  resp = requests.delete(_url("/api/leaderboard"), json=payload)
  print("Status:", resp.status_code)
  data = resp.json()
  print(data)
  return data


In [11]:
# List NEW (balanced) leaderboard
entries_new = get_leaderboard("new")
print("New (balanced) leaderboard:")
print_leaderboard(entries_new)


New (balanced) leaderboard:
id=  2 | score= 10 | name="Player" | createdAt=2026-02-23T15:06:51 | kind=new
id=  8 | score=  0 | name="testnew" | createdAt=2026-02-26T03:51:06 | kind=new


In [10]:
# List OLD leaderboard
entries_old = get_leaderboard("old")
print("Old leaderboard:")
print_leaderboard(entries_old)


Old leaderboard:
id=  3 | score=  1 | name="test1" | createdAt=2026-02-23T15:13:35 | kind=old
id=  4 | score=  0 | name="test2" | createdAt=2026-02-23T15:45:22 | kind=old
id=  5 | score=  0 | name="test3" | createdAt=2026-02-23T16:29:17 | kind=old
id=  6 | score=  0 | name="-" | createdAt=2026-02-23T19:52:21 | kind=old
id=  7 | score=  0 | name="f" | createdAt=2026-02-23T19:52:32 | kind=old


In [9]:
# Edit an entry
# 1. Run one of the listing cells above to see IDs.
# 2. Set the values below and run this cell.

entry_id = 2        # change this to the entry id you want to edit
new_name = "Player"  # new name
new_score = 10       # new score
new_kind: Kind = "new"  # "new" or "old"

edit_entry(entry_id, name=new_name, score=new_score, kind=new_kind)


Status: 200
{'ok': True}


{'ok': True}

In [ ]:
# Delete an entry
# 1. Run one of the listing cells above to see IDs.
# 2. Set the value below and run this cell.

entry_id_to_delete = 5  # change this to the entry id you want to delete

delete_entry(entry_id_to_delete)
